**In This file we perform end to end ANN and finalize the model with best weights**

In [5]:
import pandas as pd
import numpy as np
import sklearn

In [6]:
df = pd.read_csv("/content/Iris.csv")
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [7]:
df = df.drop("Id" , axis = 1)

In [8]:
df["Species"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [9]:
df["Species"] = df["Species"].map({'Iris-setosa' : 0 , 'Iris-versicolor' : 1 , 'Iris-virginica' : 2})

In [10]:
X = df.iloc[: , :-1]
y = df.iloc[: , -1]

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state=42)

In [13]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation
from tensorflow.keras.activations import relu , softmax

In [14]:
y_train_perfect = tensorflow.keras.utils.to_categorical(y_train , num_classes= len(df["Species"].unique()))

In [15]:
y_train_perfect[:5 , :]

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [16]:
import warnings
warnings.filterwarnings("ignore")

## **Architecture**

In [17]:
X_train.shape[1]

4

In [18]:
y_train_perfect.shape[1]

3

In [19]:
model = Sequential()
model.add(Dense(units = 100 , activation = relu , kernel_initializer='he_uniform' , input_dim = X_train.shape[1]))

model.add(Dense(units = 80 , activation = relu , kernel_initializer='he_uniform'))
model.add(Dense(units = 40 , activation = relu , kernel_initializer='he_uniform'))
model.add(Dense(units = 20 , activation = relu , kernel_initializer='he_uniform'))
model.add(Dense(units = 10 , activation = relu , kernel_initializer='he_uniform'))
model.add(Dense(units = 5 , activation = relu , kernel_initializer='he_uniform'))

model.add(Dense(units = y_train_perfect.shape[1] , activation= softmax , kernel_initializer= "glorot_uniform"))

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 100)                 │             500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 80)                  │           8,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 40)                  │           3,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 20)                  │             820 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 10)                  │             210 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │              55 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 3)                   │              18 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,923 (50.48 KB)

 Trainable params: 12,923 (50.48 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(optimizer="adam" , loss = "categorical_crossentropy" , metrics = ["Acc"])

we build the architecture

**now we need to train the model with data**

In [22]:
# callback function to pick best weights to finalize the model instead of taking the weigths from final epoch.
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint
checkpoint = ModelCheckpoint(
    filepath="best_model.keras",          # Filepath to save the model
    monitor='Acc',               # Metric to monitor (e.g., val_loss or val_accuracy)
    save_best_only=True,              # Save only if the current epoch is the best
    mode='max',                       # 'min' for loss, 'max' for accuracy
    verbose=1
)


In [23]:
model.fit(X_train , y_train_perfect , batch_size = 10 , epochs = 50 , validation_split = 0.2 , callbacks= checkpoint)

Epoch 1/50
 1/10 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - Acc: 0.4000 - loss: 6.1980
Epoch 1: Acc improved from -inf to 0.28125, saving model to best_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - Acc: 0.3288 - loss: 3.6483 - val_Acc: 0.5000 - val_loss: 0.9351
Epoch 2/50
 1/10 ━━━━━━━━━━━━━━━━━━━━ 2s 325ms/step - Acc: 0.6000 - loss: 0.8371
Epoch 2: Acc did not improve from 0.28125
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Acc: 0.3601 - loss: 1.0257 - val_Acc: 0.5833 - val_loss: 0.7494
Epoch 3/50
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - Acc: 0.6000 - loss: 1.0812
Epoch 3: Acc improved from 0.28125 to 0.54167, saving model to best_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - Acc: 0.5714 - loss: 0.9659 - val_Acc: 0.6667 - val_loss: 0.6226
Epoch 4/50
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - Acc: 0.5000 - loss: 1.1649
Epoch 4: Acc improved from 0.54167 to 0.64583, saving model to best_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - Acc: 0.6159 - loss: 0.9462 - val_Acc: 0.7083

training Completed.

Now we need to predict the values for test data and get the accuracy.

In [24]:
y_pred = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


In [25]:
d = []
for i in y_pred :
  d.append(np.argmax(i))

In [26]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

In [27]:
accuracy_score(y_test , d)

0.9666666666666667

In [28]:
confusion_matrix(y_test , d)

array([[10,  0,  0],
       [ 0,  8,  1],
       [ 0,  0, 11]])

In [29]:
print(classification_report(y_test , d))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.89      0.94         9
           2       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30

